In [1]:
#Goal- take blurred images and predict what digit that image is.

import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt # pyplot is for showing images

In [2]:
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [3]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
#splitting the data into training and validation data
data = np.array(data)
m , n = data.shape             #m - amount of rows , n - amounts of colmnuns(features)+1
np.random.shuffle(data)        # shuffle before splitting into dev and training sets

split_index = int(m * 0.8)
# Transpose the data slice from index 1 to 999 to prepare it for development and training sets
data_val = data[split_index:m].T
y_dev = data_val[0]            # Extract the first row as the development set labels
x_dev = data_val[1:n]          # Extract the remaining rows as the development set features


# Repeat the process for the training set, although it seems redundant since data_train is identical to data_val
data_train = data[1:split_index].T
y_train = data_train[0]
x_train = data_train[1:n]



In [5]:
x_train[:,0].shape

(784,)

In [6]:
def init_parameter():
    W1 = np.random.rand(10,784)-0.5
    B1 = np.random.rand(10,1)-0.5
    W2 = np.ranodom.rand(10,10)-0.5
    B2 = np.random.rand(10,1)-0.5

#Apply the Rectified Linear Unit (ReLU) activation function to the input z.
# This function sets all negative elements of Z to 0 and keeps positive elements unchanged.
def RelU(z):
    return np.maximum(z)

# Calculate the exponential of each element in the input array Z.
def SoftMax(z):
    A =  np.exp(Z) / sum(np.exp(Z))
#returning the array of probabilities 
    return A

def forward_propagation(W1,B1,W2,B2,X):
    # Calculate the input signal for the first layer of neurons
    Z1 = W1.dot(X) + b1
    # Apply the ReLU function to determine the activity of the first layer
    A1 = ReLU(Z1)
    # Calculate the input signal for the second layer of neurons
    Z2 = W2.dot(A1) + b2
    # Apply the softmax function to get probabilities for the second layer
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2
    
    
    
    